In [1]:
from data.dataset import REFLACXWithClinicalDataset
from model.xami import XAMIMultiModal
from libauc.optimizers import PESG
import torch.nn as nn
import torch.optim as optim
import torch 
from utils.train import split_dataset, train, train_with_auc, RecordUnit

In [2]:
# checking if the GPU is available
use_gpu = torch.cuda.is_available()

# setting up the device
device = 'cuda' if use_gpu else 'cpu'

print(f"Will be using {device}")

Will be using cuda


In [3]:
# load the dataset
reflacx_dataset = REFLACXWithClinicalDataset(image_size=256)

# keep the architecture simple, so we check the image nn first.
# prepare the model
xami_mutlimodal = XAMIMultiModal(reflacx_dataset, device, use_clinical=True)
xami_mutlimodal = xami_mutlimodal.to(device)


Positive Loss weight:
[0.9940653  0.78635013 0.8338279  0.768546   0.9985163  0.90949553
 0.99703264 0.87388724 0.99554896 0.7166172  0.99554896 0.9703264
 0.9940653  0.98219585 0.98961425 0.9718101  0.98367953 0.9376855
 0.7937685 ]
Negative Loss weight:
[0.00593472 0.21364985 0.1661721  0.231454   0.00148368 0.09050445
 0.00296736 0.12611276 0.00445104 0.2833828  0.00445104 0.02967359
 0.00593472 0.01780416 0.01038576 0.02818991 0.01632047 0.06231454
 0.20623146]
Random Loss:
0.08954620561969145


In [4]:
xami_mutlimodal.num_all_params()

7258899

In [5]:
lr = 0.0001
batch_size = 64

optimizer = optim.Adam(filter(lambda p: p.requires_grad,
                       xami_mutlimodal.parameters()), lr=lr, weight_decay=0)

# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,
#                                                  patience=1, threshold=0.001, factor=0.1)


scheduler = None


In [6]:
dataloaders = split_dataset(
    reflacx_dataset,
    batch_size,
    traing_portion=.7,
    test_portion=.15,
    seed=123
)

In [7]:
# train_data, val_data = train(
#     num_epochs=300,
#     model=xami_mutlimodal,
#     dataloaders=dataloaders,
#     scheduler=scheduler,
#     dataset=reflacx_dataset,
#     optimizer=optimizer,
#     device=device,
#     early_stop_count=None,
#     loss_weighted=True,
# )
# # if overfitting

# # Try
# # 1. dropout
# # 2. learning scheduler?


In [12]:
reflacx_dataset.df[reflacx_dataset.labels_cols].sum(axis=0)

Airway wall thickening            4
Atelectasis                     144
Consolidation                   112
Enlarged cardiac silhouette     156
Fibrosis                          1
Groundglass opacity              61
Pneumothorax                      2
Pulmonary edema                  85
Quality issue                     3
Support devices                 191
Wide mediastinum                  3
Abnormal mediastinal contour     20
Acute fracture                    4
Enlarged hilum                   12
Hiatal hernia                     7
High lung volume / emphysema     19
Interstitial lung disease        11
Lung nodule or mass              42
Pleural abnormality             139
dtype: int64

In [10]:
import numpy as np

np.array(RecordUnit.batch_target).sum(axis=0)

array([ 0, 19, 17, 25,  0,  8,  0, 14,  0, 38,  1,  5,  1,  1,  0,  2,  2,
        9, 29], dtype=int64)

In [9]:
train_data, val_data = train_with_auc(
    num_epochs=300,
    model=xami_mutlimodal,
    dataloaders=dataloaders,
    scheduler=scheduler,
    dataset=reflacx_dataset,
    device=device,
    early_stop_count=None,
)

# if overfitting

# Try

Epoch 1/300
----------
Epoch: 1 Batch:  (0) Train Loss: 0.7259
Epoch: 1 Batch:  (1) Train Loss: 0.7045
Epoch: 1 Batch:  (2) Train Loss: 0.7098
Epoch: 1 Batch:  (3) Train Loss: 0.7968
Epoch: 1 Batch:  (4) Train Loss: 0.6831
Epoch: 1 Batch:  (5) Train Loss: 0.7048
Epoch: 1 Batch:  (6) Train Loss: 0.7397
Epoch: 1 Batch:  (7) Train Loss: 0.7831
Epoch 1 | Loss: 0.73 | ACC: 46.53% | AUC: 0.52
Epoch: 1 Batch:  (0) Test Loss: 0.7561
Epoch: 1 Batch:  (1) Test Loss: 0.7650


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.